# Notes

Fourth iteration of the modeling:
- modulirized data processing
- model serialization

In [1]:
import os 
import re
import csv
import json
import math
import random

from itertools import chain
from datetime import datetime

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
plt.style.use('_mpl-gallery')

# Constants

In [2]:
dir_root = os.path.join('..')
dir_data_root = os.path.join(dir_root, 'data')
dir_exercises = os.path.join(dir_data_root, 'json', 'patients')
dir_exercises_augmented = os.path.join(dir_data_root, 'json', 'patients_augmented')
dir_exercises_test = os.path.join(dir_data_root, 'json', 'test')

**TODO** Implement augmenation and patients sessions for the new data set

# Data prep

### Build training set

In [3]:
CONFIG = {
    'SEQ_MAX_LEN': 801,
    'NUM_EXERCISES': 9,
    'NUM_CLASSES': 6,
    'REGIONS':json.load(open(os.path.join('settings', 'pois_region_tpcnn.json'))),
    'AUGMENTED': True
}

In [4]:
CONFIG

{'SEQ_MAX_LEN': 801,
 'NUM_EXERCISES': 9,
 'NUM_CLASSES': 6,
 'REGIONS': {'LefteyeMidbottom': 'upper',
  'LefteyeMidtop': 'upper',
  'LefteyeInnercorner': 'upper',
  'LefteyeOutercorner': 'upper',
  'LefteyebrowInner': 'upper',
  'LefteyebrowCenter': 'upper',
  'RighteyeMidbottom': 'upper',
  'RighteyeMidtop': 'upper',
  'RighteyeInnercorner': 'upper',
  'RighteyeOutercorner': 'upper',
  'RighteyebrowInner': 'upper',
  'RighteyebrowCenter': 'upper',
  'NoseTip': 'lower',
  'MouthLowerlipMidbottom': 'lower',
  'MouthLeftcorner': 'lower',
  'MouthRightcorner': 'lower',
  'MouthUpperlipMidtop': 'lower',
  'ChinCenter': 'lower',
  'ForeheadCenter': 'upper',
  'LeftcheekCenter': 'lower',
  'RightcheekCenter': 'lower'},
 'AUGMENTED': True}

input for global region
801 is length of a series in excercise 
each series consist from 63 value, 21 POIs, each POI has 3 values: x,y,z

overall length of a series is 7209 (in paper 8019?)

In [5]:
i = 0 
exercise_ids = set()
for file_name in os.listdir(dir_exercises):
    ids = file_name.split('.')
    exercise_id = f'{ids[0]}'
    exercise_ids.add(exercise_id)
exercise_ids = sorted(list(exercise_ids))

In [6]:
len(exercise_ids)

2419

In [7]:
def pad_excercise_poi(data, poi):
    return tf.keras.preprocessing.sequence.pad_sequences(
        [
            data['pois'][poi]['xs'],
            data['pois'][poi]['ys'],
            data['pois'][poi]['zs']
        
        ],
        padding='pre',
        maxlen=CONFIG['SEQ_MAX_LEN'],
        dtype='float32')

In [8]:
def session_data(exercise_id, augmented = None):
    global_region = None
    upper_region = None
    lower_region = None
    exercises_data = None
    
    if augmented:
        exercises_data = json.load(open(os.path.join(dir_exercises_augmented, f'{exercise_id}_{augmented}.json')))
    else:
        exercises_data = json.load(open(os.path.join(dir_exercises, f'{exercise_id}.json')))
    
    for poi in sorted(CONFIG['REGIONS'].keys()):
        poi_series = pad_excercise_poi(exercises_data, poi)
        
        if global_region is not None:
            global_region = np.concatenate((global_region, poi_series), axis=0)
        else:
            global_region = poi_series
        
        if CONFIG['REGIONS'][poi] == 'upper':
            if upper_region is not None:
                upper_region = np.concatenate((upper_region, poi_series), axis=0)
            else:
                upper_region = poi_series
        if CONFIG['REGIONS'][poi] == 'lower':
            if lower_region is not None:
                lower_region = np.concatenate((lower_region, poi_series), axis=0)
            else:
                lower_region = poi_series

    
    y = int(exercises_data['meta']['evaluation']) - 1
    

    return [
        global_region,
        upper_region,
        lower_region,
        y
    ]

In [9]:
xslist_global = list()
xslist_upper = list()
xslist_lower = list()
yslist = list()
incomlete_sessions = []
i = 0
for exercise_id in exercise_ids:
    try:
        _xs_global, _xs_upper, _xs_lower, _ys = session_data(exercise_id)
        

        yslist.append(_ys)
        xslist_global.append(_xs_global)  
        xslist_upper.append(_xs_upper)
        xslist_lower.append(_xs_lower)
    except:
        print(f'issues with session_id: {session_id}')
        incomlete_sessions.append(session_id)
        continue
    
    if CONFIG['AUGMENTED']:
        for augmentation_id in range(4):
            try:
                _xs_global, _xs_upper, _xs_lower, _ys = session_data(exercise_id, augmentation_id)


                yslist.append(_ys)
                xslist_global.append(_xs_global)  
                xslist_upper.append(_xs_upper)
                xslist_lower.append(_xs_lower)
            except:  
                break
    
ys = np.array(yslist, dtype='int')
xs_global = np.array(xslist_global, dtype='float32') 
xs_upper = np.array(xslist_upper, dtype='float32') 
xs_lower = np.array(xslist_lower, dtype='float32') 


print(ys.shape)
print(xs_global.shape)
print(xs_upper.shape)
print(xs_lower.shape)
print(f'Incomplete sessions {len(incomlete_sessions)}')

(12095,)
(12095, 63, 801)
(12095, 39, 801)
(12095, 24, 801)
Incomplete sessions 0


In [10]:
print(ys)

[0 0 0 ... 0 0 0]


In [11]:
print(set(ys))

{0, 1, 2, 3, 4, 5}


## Modeling

In [12]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.utils import class_weight

In [13]:
def get_dnn(inputLayer): 
    m = Dense(4, activation="relu")(inputLayer)
    m = Model(inputs=inputLayer, outputs=m)

    return m

In [14]:
def get_regional_cnn(inputLayer):
    
    m = Conv1D(32, 3, padding='same', activation='relu')(inputLayer)
    m = AveragePooling1D(3)(m)
    m = Conv1D(32, 3, padding='same', activation='relu')(m)
    m = Conv1D(64, 3, padding='same', activation='relu')(m)
    m = AveragePooling1D(3)(m)
    m = Conv1D(64, 3, padding='same', activation='relu')(m)
    m = Dense(128, activation="relu")(m)
    m = Dropout(0.5)(m)
    m = Flatten()(m)
    m = Model(inputs=inputLayer, outputs=m)

    return m

In [15]:
def get_global_cnn(inputLayer):    
    m = Conv1D(6, 21, padding='same', activation='relu')(inputLayer)
    m = AveragePooling1D(3)(m)
    m = Conv1D(6, 9, padding='same', activation='relu')(m)
    m = Conv1D(16, 9, padding='same', activation='relu')(m)
    m = AveragePooling1D(3)(m)
    m = Conv1D(16, 3, padding='same', activation='relu')(m)
    m = Dense(128, activation="relu")(m)
    m = Dropout(0.5)(m)
    m = Flatten()(m)
    m = Model(inputs=inputLayer, outputs=m)

    return m

In [16]:
def get_model():
    input_global = Input(shape=xs_global.shape[1:])
    model_global = get_global_cnn(input_global)
    
    input_upper = Input(shape=xs_upper.shape[1:])
    model_upper = get_regional_cnn(input_upper)  

    input_lower = Input(shape=xs_lower.shape[1:])
    model_lower = get_regional_cnn(input_lower)  
    
    
    model_contatenate = concatenate([
        model_global.output,
        model_upper.output,
        model_lower.output,
    ])
    
    model_contatenate = Dense(6, activation="softmax")(model_contatenate)
        
    model = Model(inputs=[
        model_global.input,
        model_upper.input,
        model_lower.input
    ], outputs=model_contatenate)

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer=Adam(learning_rate=1e-3, decay=1e-3 / 200),
        metrics=['accuracy']
    )
    
    
    return model

In [17]:
test_model = get_model()
test_model.summary()

Metal device set to: Apple M1


2024-03-03 09:19:37.768381: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-03 09:19:37.768743: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 63, 801)]    0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 39, 801)]    0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 24, 801)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 63, 6)        100932      ['input_1[0][0]']                
                                                                                            

In [18]:
print(set(ys))

{0, 1, 2, 3, 4, 5}


### K-fold evaluation

In [19]:
k = 5
k_limit = 5
train = 0.8
val = 0.2
test = 0.2

In [20]:
VERBOSE = 0
EPOCHS = 400
BATCH_SIZE = 64

In [21]:
from sklearn.model_selection import KFold

In [22]:
def get_k_indx(k, n):

    k_fold = KFold(n_splits=k)
    train_ = []
    val_ = []
    test_ = []
    indx = []

    for train_indices, test_indices in k_fold.split(ys):
        n_k = len(train_indices)
        val_split = int(n_k * train)
        indx.append([train_indices[:val_split],train_indices[val_split + 1:], test_indices])
    
    return indx

In [23]:
%matplotlib inline

In [ ]:
indxs = get_k_indx(k, len(ys))
models = []

fig, axs = plt.subplots(1,5,figsize=(10,2)) 
colors = ['red', 'pink', 'blue', 'cyan']
labels = ['Loss training', 'Loss validation', 'Accuracy training', 'Accuracy validation']

for i in range(k_limit):
    train_indx, val_indx, test_indx  = indxs[i]
    
    xs_global_i = xs_global[train_indx]
    xs_global_i_val = xs_global[val_indx]
    xs_global_i_test = xs_global[test_indx]
    
    xs_upper_i = xs_upper[train_indx]
    xs_upper_i_val = xs_upper[val_indx]
    xs_upper_i_test = xs_upper[test_indx]
    
    xs_lower_i = xs_lower[train_indx]
    xs_lower_i_val = xs_lower[val_indx]
    xs_lower_i_test = xs_lower[test_indx]
    
    ys_i = ys[train_indx]
    ys_i_val = ys[val_indx]
    ys_i_test = ys[test_indx]
    
    
    class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes = np.unique(ys_i),
                                                 y=ys_i)
    available_classes = np.unique(ys_i)
    weight_training_classes = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
    }
    
    for indx, value in enumerate(class_weights):
        weight_training_classes[available_classes[indx]] = value

    #print(weight_training_classes)

    model = get_model()

    model_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20)
    ]
    
    hist = model.fit(
        x=[
            xs_global_i, 
            xs_upper_i,
            xs_lower_i], y=ys_i, 
        validation_data=([
            xs_global_i_val,
            xs_upper_i_val,
            xs_lower_i_val], ys_i_val),
        batch_size=BATCH_SIZE, 
        epochs=EPOCHS,
        class_weight=weight_training_classes,
        #callbacks=model_callbacks,
        verbose=VERBOSE)
    
    models.append(model)
    
    # visualizing losses and accuracy
    train_loss = hist.history['loss']
    val_loss   = hist.history['val_loss']
    train_acc  = hist.history['accuracy']
    val_acc    = hist.history['val_accuracy']
    xc         = range(len(hist.history['loss']))    
    
    axs[i].set_title(f'Run {i}')
    axs[i].plot(xc, train_loss, color=colors[0], label=labels[0])
    axs[i].plot(xc, val_loss, color=colors[1], label=labels[1])
    axs[i].plot(xc, train_acc, color=colors[2], label=labels[2])
    axs[i].plot(xc, val_acc, color=colors[3], label=labels[3])
    
    y_pred = model.predict([
        xs_global_i_test,
        xs_upper_i_test,
        xs_lower_i_test],verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)

    print(classification_report(ys_i_test, y_pred_bool))
    
    
fig.legend(labels, ncol=1, bbox_to_anchor=(1.2,0.5), loc="center right", fontsize=10, 
           bbox_transform=plt.gcf().transFigure)

plt.show()
fig.savefig(f'tpcnn_exercise_runs_01-{int(time.time())}.svg', format='svg', dpi=1200)   

2024-03-03 09:19:42.436336: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-03 09:19:43.265218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 09:19:49.472265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 09:57:13.918406: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.44      0.36      0.40      1069
           1       0.17      0.22      0.19       315
           2       0.18      0.36      0.24       225
           3       0.00      0.00      0.00       225
           4       0.02      0.02      0.02       270
           5       0.29      0.20      0.24       315

    accuracy                           0.25      2419
   macro avg       0.18      0.19      0.18      2419
weighted avg       0.27      0.25      0.26      2419



2024-03-03 09:57:26.824348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 09:57:35.518906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 10:40:32.358539: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.46      0.45      0.46      1134
           1       0.18      0.18      0.18       210
           2       0.09      0.12      0.10       225
           3       0.03      0.03      0.03       190
           4       0.27      0.27      0.27        90
           5       0.23      0.22      0.22       570

    accuracy                           0.30      2419
   macro avg       0.21      0.21      0.21      2419
weighted avg       0.31      0.30      0.30      2419



2024-03-03 10:40:47.859053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 10:40:57.514785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 11:31:23.957962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.32      0.53      0.40       749
           1       0.14      0.10      0.12       490
           2       0.30      0.14      0.19       565
           3       0.00      0.00      0.00        45
           4       0.42      0.19      0.26       225
           5       0.22      0.21      0.21       345

    accuracy                           0.26      2419
   macro avg       0.23      0.19      0.20      2419
weighted avg       0.27      0.26      0.25      2419



2024-03-03 11:31:44.065031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 11:31:57.711638: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 12:22:00.991797: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/alanchen/.local/share/virtualenvs/research-face-excercises-dynamics-o_HFBNSW/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alanchen/.local/share/virtualenvs/research-face-excercises-dynamics-o_HFBNSW/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: Undefi

              precision    recall  f1-score   support

           0       0.57      0.59      0.58      1328
           1       0.09      0.06      0.07       415
           2       0.15      0.11      0.13       421
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00        45
           5       0.11      0.17      0.13       210

    accuracy                           0.37      2419
   macro avg       0.15      0.16      0.15      2419
weighted avg       0.36      0.37      0.36      2419



2024-03-03 12:22:19.686323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 12:22:30.662240: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


# Model Serialization 

In [ ]:
serialize = False
best_model = 2

In [ ]:
dir_models = os.path.join(dir_root, 'models')
dir_model = os.path.join(dir_models, 'type4-with-fe_no-aug_005')

In [ ]:
if serialize:
    model = models[best_model]
    model.save(dir_model)